In [2]:
import pandas as pd
from dateutil import parser
import numpy as np
import math

In [3]:
df = pd.read_excel('Lista Incendios_2015.xlsx')

In [9]:
x = df.iloc[0:10,12]
y = [str(x[0]),str(x[1])]
#print(y[0][0:7] > '8:19:0')
print(x)

0    8:31:12.3276000000027''
1    8:12:28.3788000000025''
2     8:28:1.8084000000006''
3    8:28:41.6172000000014''
4    7:55:40.1268000000016''
5    8:41:58.0343999999977''
6                   07:54:43
7    7:59:7.30320000000106''
8    8:40:17.8248000000018''
9    7:42:11.7215999999988''
Name: lon, dtype: object


In [4]:
tempo_intervencao = [None] * (len(df.index))
tempo_extincao = [None] * (len(df.index))
litoralidade = [None] * (len(df.index))
for i in range(len(df.index)):
    long = str(df.iloc[i,12])
    if long[0:7] >= '8:19:0':
        litoralidade[i] = 'Litoral'
    if long[0:7] < '8:19:0':
        litoralidade[i] = 'Interior'
    if pd.isna(df.iloc[i,13]) or pd.isna(df.iloc[i,14]) or pd.isna(df.iloc[i,17]) or pd.isna(df.iloc[i,18]):
        tempo_intervencao[i] = ('Cannot calcule')
    if pd.isna(df.iloc[i,15]) or pd.isna(df.iloc[i,16]) or pd.isna(df.iloc[i,17]) or pd.isna(df.iloc[i,18]):
        tempo_extincao[i] = ('Cannot calcule')
    else: 
        data_alerta = parser.parse(str(df.iloc[i,13][:10]) + ' ' + str(df.iloc[i,14]))
        data_int = parser.parse(str(df.iloc[i,17][:10]) + ' ' + str(df.iloc[i,18]))
        data_ext = parser.parse(str(df.iloc[i,15][:10]) + ' ' + str(df.iloc[i,16]))
        tempo_intervencao[i] = (str(abs(data_int - data_alerta)))
        tempo_extincao[i] = (str(abs(data_ext - data_int)))

#df['TempoExtincao'] = tempo_extincao
#df['TempoIntervencao'] = tempo_intervencao
#df.loc[df['Freguesia'] == 'Perelhal']
df.insert(15, "TempoExtincao", tempo_extincao , True)
df.insert(16, "TempoIntervencao", tempo_intervencao , True)
df.insert(17, "Litoralidade", litoralidade , True)
df.loc[df['Litoralidade'] == 'Litoral']

,Ano,Codigo SGIF,Codigo_ANPC,Tipo,Distrito,Concelho,Freguesia,Local,INE,x,...,Falso Alarme,Fogacho,Incendio,Agricola,Perimetro,APS,Causa,TipoCausa,Região PROF,UGF
0,2015,DM315252,2015160007544,Florestal,Viana do Castelo,Ponte de Lima,Serdedelo,VALE DE TROVELA (SERDEDELO),160746,167814,...,0,0,1,0,NaN,NaN,122.0,Negligente,Entre Douro e Minho,Alto Minho
2,2015,DM415293,2015030021973,Florestal,Braga,Vila Verde,Lage,RUA PROF ABEL MADEIRA,31323,172162,...,0,1,0,0,NaN,NaN,60.0,Desconhecida,Entre Douro e Minho,Baixo Minho
3,2015,DM315261,2015160007563,Falso Alarme,Viana do Castelo,Ponte de Lima,Boalhosa,J. DEPOSITOS DE AGUA (BOALHOSA),160709,171291,...,1,0,0,0,NaN,NaN,630.0,Desconhecida,Entre Douro e Minho,Alto Minho
5,2015,DM415289,2015030021952,Florestal,Braga,Barcelos,Perelhal,ERMIDA,30260,152733,...,0,0,1,0,NaN,NaN,711.0,Reacendimento,Entre Douro e Minho,Baixo Minho
8,2015,DM215334,2015130044169,Florestal,Porto,Matosinhos,Santa Cruz do Bispo,MONTE DA MINA,130808,154833,...,0,1,0,0,NaN,NaN,NaN,NaN,Entre Douro e Minho,AMP e Entre Douro e Vouga
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23169,2015,RO115892,2015140044046,Florestal,Santarém,Ourém,Nossa Senhora Misericórdias,Lagoa do Furadouro,142111,162243,...,0,1,0,0,NaN,NaN,448.0,Intencional,Ribatejo e Oeste,Ribatejo
23171,2015,BL1151018,2015180041997,Florestal,Viseu,Cinfães,Cinfães,Ventuzela,180403,203110,...,0,0,1,0,NaN,NaN,125.0,Negligente,Entre Douro e Minho,Tâmega
23172,2015,BL3151227,2015010055691,Falso Alarme,Aveiro,Santa Maria da Feira,Feira,avenida 25 de abril,10906,164395,...,1,0,0,0,NaN,NaN,NaN,NaN,Entre Douro e Minho,AMP e Entre Douro e Vouga
23173,2015,DM2153226,2015130115988,Florestal,Porto,Paços de Ferreira,Paços de Ferreira,Lugar alto das Cavadas,130912,180666,...,0,1,0,0,NaN,NaN,711.0,Reacendimento,Entre Douro e Minho,Tâmega


In [8]:
df.iloc[:,15:18]

,TempoExtincao,TempoIntervencao,Litoralidade
0,0:59:00,0:09:00,Litoral
1,1:31:00,0:06:00,Interior
2,0:35:00,0:10:00,Litoral
3,1:10:00,0:15:00,Litoral
4,0:45:00,0:05:00,Interior
...,...,...,...
23170,0:55:00,0:10:00,Interior
23171,4:37:00,0:13:00,Litoral
23172,0:45:00,0:05:00,Litoral
23173,2:15:00,0:05:00,Litoral


In [5]:
reacendimentos = df.loc[df['TipoCausa'] == 'Reacendimento']
nao_reacendimentos = df.loc[df['TipoCausa'] != 'Reacendimento']
a_reacendimentos = reacendimentos.iloc[:,23:28]
a_nao_reacendimentos = nao_reacendimentos.iloc[:,23:28]
#print(a_reacendimentos.describe())
#print(a_nao_reacendimentos.describe())
reacendimentos['Count'] = 1
reac = reacendimentos.groupby('Freguesia').count()
print(reac.loc[reac['Count'] > 5])

                          Ano  Codigo SGIF  Codigo_ANPC  Tipo  Distrito  \
Freguesia                                                                 
Aboadela                    7            7            7     7         7   
Arcos                       6            6            6     6         6   
Ardegão                     7            7            7     7         7   
Avioso (Santa Maria)        9            9            9     9         9   
Avioso (São Pedro)          8            8            8     8         8   
...                       ...          ...          ...   ...       ...   
Vila Franca                 6            6            6     6         6   
Vilela                     14           14           14    14        14   
Vinhó                       7            7            7     7         7   
Vitorino dos Piães          7            7            7     7         7   
Várzea Ovelha e Aliviada    6            6            6     6         6   

                        

/var/folders/zb/q2q0b14j1wx8tdztc_894g9c0000gn/T/ipykernel_17334/2971690799.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reacendimentos['Count'] = 1
